In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master("local[*]").appName('test').getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/04/16 08:10:07 WARN Utils: Your hostname, MSI resolves to a loopback address: 127.0.1.1; using 172.24.150.144 instead (on interface eth0)
23/04/16 08:10:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/16 08:10:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/16 08:10:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df_yellow = spark.read.parquet('data/report/revenue/yellow/')
df_green = spark.read.parquet('data/report/revenue/green/')

In [4]:
df_zone = spark.read.parquet('zones/')

In [5]:
df_zone.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

#### join 2 dataframe - both similar sizes

In [6]:
df_yellow.show(5)

+-------------------+------------+------------------+---------+
|               hour|PULocationID|            amount|num_trips|
+-------------------+------------+------------------+---------+
|2020-11-10 14:00:00|         163|1802.9599999999987|      117|
|2020-11-10 14:00:00|         164|           1329.32|       93|
|2020-11-10 14:00:00|         166| 427.1400000000001|       26|
|2020-11-10 14:00:00|         167|               3.3|        1|
|2020-11-10 14:00:00|         168|30.900000000000002|        3|
+-------------------+------------+------------------+---------+
only showing top 5 rows



In [7]:
df_green.show(5)

+-------------------+------------+------------------+---------+
|               hour|PULocationID|            amount|num_trips|
+-------------------+------------+------------------+---------+
|2020-07-01 20:00:00|          41|             40.11|        3|
|2020-07-01 20:00:00|          42|29.400000000000002|        3|
|2020-07-01 20:00:00|          43|60.519999999999996|        4|
|2020-07-01 20:00:00|          49|              15.0|        1|
|2020-07-01 20:00:00|          51|              32.5|        1|
+-------------------+------------+------------------+---------+
only showing top 5 rows



In [8]:
df_green = df_green.withColumnRenamed('amount', 'green_amount')
df_green = df_green.withColumnRenamed('num_trips', 'green_num_trips')
df_yellow = df_yellow.withColumnRenamed('num_trips', 'yellow_num_trips')
df_yellow = df_yellow.withColumnRenamed('amount', 'yellow_amount')

In [13]:
df_joined = df_green.join(df_yellow,on=['hour','PULocationID'],how='outer')

In [15]:
df_joined.show(50)

+-------------------+------------+------------------+---------------+------------------+----------------+
|               hour|PULocationID|      green_amount|green_num_trips|     yellow_amount|yellow_num_trips|
+-------------------+------------+------------------+---------------+------------------+----------------+
|2020-01-01 00:00:00|          14|              null|           null|               8.8|               1|
|2020-01-01 00:00:00|          22|              15.8|              1|              null|            null|
|2020-01-01 00:00:00|          45|              null|           null| 732.4800000000001|              42|
|2020-01-01 00:00:00|          51|              17.8|              2|              31.0|               1|
|2020-01-01 00:00:00|          62|             15.95|              1|             61.43|               1|
|2020-01-01 00:00:00|          72|             67.53|              2|              null|            null|
|2020-01-01 00:00:00|          89|            

In [18]:
df_joined.write.parquet('data/report/revenue/total')

23/04/16 08:22:09 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


#### join 2 dataframe - one small one big

In [20]:
df_join = spark.read.parquet('data/report/revenue/total/')

In [21]:
df_join.join(df_zone, df_join.PULocationID==df_zone.LocationID).show()

+-------------------+------------+------------------+---------------+------------------+----------------+----------+---------+--------------------+------------+
|               hour|PULocationID|      green_amount|green_num_trips|     yellow_amount|yellow_num_trips|LocationID|  Borough|                Zone|service_zone|
+-------------------+------------+------------------+---------------+------------------+----------------+----------+---------+--------------------+------------+
|2020-01-01 00:00:00|          24|              87.6|              3| 754.9500000000003|              45|        24|Manhattan|        Bloomingdale| Yellow Zone|
|2020-01-01 00:00:00|          40|168.98000000000002|              8|             89.97|               5|        40| Brooklyn|     Carroll Gardens|   Boro Zone|
|2020-01-01 00:00:00|          47|              13.3|              1|               8.3|               1|        47|    Bronx|  Claremont/Bathgate|   Boro Zone|
|2020-01-01 00:00:00|          49|